In [1]:
df=spark.sql("select * from user_erin.food_features")
(train, test) = df.randomSplit([0.8, 0.2],seed = 11)

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
1,None,pyspark,idle,Link,,erin,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## 机器学习：线性回归模型的调参与预测
- 线性回归通过使用最佳的拟合直线（又被称为回归线），建立因变量（Y）和一个或多个自变量（X）之间的关系。它的表达式为：Y=a+b*X+e，其中 a 为直线截距，b 为直线斜率，e 为误差项。如果给出了自变量 X，就能通过这个线性回归表达式计算出预测值，即因变量 Y。
- 运用TrainValidationSplit（TVS）来进行参数调优。TVS对于每个参数组合只进行一次评估，当训练数据集足够大时，开销较小，并且产生的结果可靠。TVS会创建单个 (training, test) pair。它使用trainRatio参数将数据集split成两部分。例如： trainRatio=0.8，TVS会生成一个训练集（80%）和一个测试集（20%）。TVS最后会使用Estimator、以及最好的ParamMap，对整个数据集进行拟合。


In [ ]:
from pyspark.ml.tuning import ParamGridBuilder, TrainValidationSplit
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator
lr = LinearRegression(featuresCol = 'newnfeatures', labelCol='score')
lr_evaluator = RegressionEvaluator(predictionCol="prediction", \
                 labelCol="score",metricName="r2")
param_grid = ParamGridBuilder() \
            .addGrid(lr.regParam, [ .01, .05, .1, .15]) \
            .addGrid(lr.elasticNetParam, [ 0.0, 0.5, 1.0]) \
            .build()
tvs = TrainValidationSplit(estimator=lr,
                           estimatorParamMaps=param_grid,
                           evaluator=lr_evaluator,
                           trainRatio = 0.8)
model=tvs.fit(train)
lr_prediction=model.transform(test)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
lr_prediction=spark.sql("select * from user_erin.food_lrp")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
from pyspark.ml.evaluation import RegressionEvaluator
lr_evaluator = RegressionEvaluator(predictionCol="prediction", \
                 labelCol="score",metricName="mae")
result=lr_evaluator.evaluate(lr_prediction)
print("mae result for prediction is",result)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

rmse result for prediction is 0.707833791429223

In [ ]:
lr_prediction.show(3)

In [8]:
lr_prediction.write.saveAsTable("user_erin.food_lrp", format="orc", mode="overwrite")
model.write().overwrite().save("hdfs:///user/erin/data/mllib/testModelPath") 

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
spark.stop()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…